In [ ]:
#Example code for OpenAI LLM calling

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()             
client = OpenAI()        

resp = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are concise."},
        {"role": "user", "content": "Say hi in one sentence."},
    ],
)
print(resp.choices[0].message.content)


Hi there!


In [ ]:
import json
from openai import OpenAI
client = OpenAI()

#Define the tool (function schema)
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temp in a city",
        "parameters": {
            "type":"object",
            "properties":{"city":{"type":"string"}},
            "required":["city"]
        }
    }
}]

#REPLACE WITH REAL FUNCTION HERE FOR TOOLING
def get_weather(city: str):
    return {"city": city, "tempC": 6767676767676767676767}

messages = [{"role":"user","content":"What's the temp in Auckland?"}]
resp = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    tool_choice="auto",
)

msg = resp.choices[0].message
tool_calls = msg.tool_calls or []

if tool_calls:
    messages.append({
        "role": "assistant",
        "tool_calls": [
            {
                "id": tool_calls[0].id,
                "type": "function",
                "function": {
                    "name": tool_calls[0].function.name,
                    "arguments": tool_calls[0].function.arguments
                }
            }
        ]
    })

    args = json.loads(tool_calls[0].function.arguments)
    result = get_weather(**args)

    messages.append({
        "role": "tool",
        "tool_call_id": tool_calls[0].id,
        "content": json.dumps(result)
    })

    final = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    print(final.choices[0].message.content)
else:
    print(msg.content)


The current temperature in Auckland is 17.2°C.
